In [159]:
import pandas as pd
import ast
from sklearn.metrics.pairwise import pairwise_distances
import numpy as np


df = pd.read_csv('movies_dataset_modificado.csv', low_memory=False)


In [160]:
type(df.at[1,"name_Genres"])

str

In [161]:
df["name_Genres"] = df["name_Genres"].astype(str)


In [162]:
df["name_Genres"] = df["name_Genres"].apply(lambda x: x.replace("[", "").replace("]", "").replace("'", "").replace(", ", ","))


In [163]:
data = df[['title', 'vote_average', 'name_Genres', "release_year", "name_production_companies","overview"]]

In [164]:
type(df.at[4,"release_year"])

numpy.float64

In [165]:
#prueba utilizada
df1 = df.head(30000).reset_index(drop=True)
titles = df1['title']
titles

0                                Toy Story
1                                  Jumanji
2                         Grumpier Old Men
3                        Waiting to Exhale
4              Father of the Bride Part II
                       ...                
29995                      Seventeen Again
29996                        Sweet Sixteen
29997    The Disappearance of Garcia Lorca
29998              The Dramatics: A Comedy
29999                         Up the Creek
Name: title, Length: 30000, dtype: object

In [166]:
similarity_matrix =  pairwise_distances(data.pivot_table(index='name_Genres', columns=['release_year']).fillna(0), metric="cosine")

C:\Users\tonto\AppData\Local\Temp\ipykernel_16728\3122387956.py:1: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  similarity_matrix =  pairwise_distances(data.pivot_table(index='name_Genres', columns=['release_year']).fillna(0), metric="cosine")


In [167]:
similarity_matrix = pairwise_distances(data.pivot_table(index='name_Genres', columns='release_year', values='vote_average', fill_value=0), metric='cosine')


In [168]:
def recomendacion(titulo):
    
    
   
    
    # Buscar la película seleccionada por el usuario en el DataFrame
    idx = data[data['title'] == titulo].index[0]
    
    # Ordenar la matriz de similaridad para obtener las películas más similares
    similar_movie_indices = np.argsort(similarity_matrix[idx])
    
    # Seleccionar las primeras 5 películas más similares y obtener sus títulos
    recommended_movies = []
    for i in similar_movie_indices:
        if data.iloc[i]['title'] != titulo:
            recommended_movies.append(data.iloc[i]['title'])
        if len(recommended_movies) == 5:
            break
    
    # Devolver una lista con los títulos de las películas recomendadas
    return {'lista recomendada': recommended_movies}


In [169]:
recomendacion(("The Exorcist III"))


{'lista recomendada': ['The Yards',
  "Rosemary's Baby",
  'Best Men',
  'Lured',
  'Farinelli']}

In [171]:
pelicula_entrada = data.loc[data['title'] ==   "Rosemary's Baby"]
generos_entrada = pelicula_entrada['name_Genres'].values[0]

generos_entrada

'Horror,Drama,Mystery'

In [172]:
pelicula_entrada = data.loc[data['title'] ==   'Farinelli']
generos_entrada = pelicula_entrada['release_year'].values[0]

generos_entrada

1994.0

In [173]:
print(data['release_year'].isnull().sum())

90
